## Exponential Moving Average
t_i = alpha * t_{i-1} + (1 - alpha) * s_i, with a value of alpha = 0.99

In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))

In [2]:
import numpy as np
from src.model import mean_teacher

Using TensorFlow backend.


In [3]:
configs = {
    'input_shape': (480,640,3),
    'num_of_classes': 10
}

In [4]:
student_model, teacher_model = mean_teacher(configs)

Instructions for updating:
Colocations handled automatically by placer.


/home/dh/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [21]:
def ema(student_model, teacher_model, alpha = 0.99):
    '''
    Calculates the exponential moving average of the student model weights and updates the teacher model weights\
    
    formula:
    t_i = alpha * t_{i-1} + (1 - alpha) * s_i, with default alpha = 0.99
    t_i = weights of teacher model in current epoch
    s_i = weights of student model in current epoch
    
    '''
    
    student_weights = student_model.get_weights()
    teacher_weights = teacher_model.get_weights()
    
    assert len(student_weights) == len(teacher_weights), 'length of student and teachers weights are not equal Please check. \n Student: {}, \n Teacher:{}'.format(len(student_weights), len(teacher_weights))
    
    new_layers = []
    for i, layers in enumerate(student_weights):
        new_layer = alpha*(teacher_weights[i]) + (1-alpha)*layers
        new_layer.trainable = False
        new_layers.append(new_layer)
    teacher_model.set_weights(new_layers)
    

## Loss function
### model architecture

![test](pictures/student_teacher_model_arch.png)

In [22]:
from keras.optimizers import Adam
from keras.models import Model
import keras.backend as K

In [12]:
def categorical_crossentropy(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred)

In [13]:
def weighted_sum_loss(squared_difference_layer):

    def categorical_crossentropy_custom(y_true, y_pred):
        ratio = 0.5
        return ratio * K.categorical_crossentropy(y_true, y_pred) + (1 - ratio)*squared_difference_layer
    
    return categorical_crossentropy_custom

In [23]:
x_src = Input()
x_tgt = Input()
x_src_student = student_model(x_src)
x_tgt_student = student_model(x_tgt)
x_tgt_teacher = teacher_model(x_tgt)
# outputs = K.square(student_model.output - teacher_model.output)
sq_diff = K.sum(K.square( x_tgt_student - x_tgt_teacher ))

def custom_loss(y_true, y_pred):
    ratio = 0.5
    return ratio * K.categorical_crossentropy(y_true, y_pred) + (1 - ratio)*sq_diff


Model([x_src, x_tgt], x_src_student)

useful links:
- https://keras.io/layers/about-keras-layers/
- https://keras.io/getting-started/functional-api-guide/
- https://github.com/keras-team/keras/blob/master/keras/losses.py
- https://keras.io/getting-started/functional-api-guide/
- https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618